---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd
import re

In [2]:
def add_slash(val,s):
    if s in val:
        return val.replace(s,'/')
    else:
        return val
    
def add_nineteen(val):
    exts = val.split('/')
    if int(exts[-1])/100 < 1:
        temp = str(19)+exts[-1]
        ans = exts[0].strip() + '/' + exts[1].strip() + '/' + temp.strip()
    else:
        ans = val
    return ans

def clean_punct(val):
    if '.' in val or ',' in val:
        if '.' in val:
            val =  val.replace('.','')
        if ',' in val:
            ans = val.replace(',','')
    else:
        ans = val
    return ans

def t2_months(val):
    exts = val.split('/')
    ans = get_mo(exts[1]) + '/' + exts[0].strip() + '/' + exts[-1].strip()
    return ans
    
def t3_months(val):
    exts = val.split('/')
    ans = get_mo(exts[0]) + '/' + exts[1].strip() + '/' + exts[-1].strip()
    return ans

def t4_months(val):
    exts = val.split('/')
    ans = get_mo(exts[0]) + '/' + str(1) + '/' + exts[-1].strip()
    return ans

def t5_months(val):
    exts = val.split('/')
    ans = exts[0] + '/' + str(1) + '/' + exts[-1].strip()
    return ans

def t6_months(val):
    ans = str(1) + '/' + str(1) + '/' + val.strip()
    return ans

def get_mo(val):
    months = {'January':'01', 'Janaury':'01', 'Jan' : '01', \
              'February':"02", 'Feb' : '02', \
              'March' : '03', 'Mar' : '03', \
              'April': '04', 'Apr' : '04', \
              'May' : '05', \
              'June' : '06', 'Jun' : '06', \
              'July' : '07', 'Jul' : '07', \
              'August' : '08' , 'Aug' : '08' , \
              'September' : '09' , 'Sep' : '09',\
              'October' : '10', 'Oct' : '10', \
              'November' : '11', 'Nov' : '11',\
              'December': '12', 'Decemeber' : '12' , 'Dec': '12'}
    if val in months.keys():
        return months[val]
    else:
        return False

In [3]:
def type1(val):
    type1 = re.compile(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}')
    ans = re.findall(type1,val)
    ans = add_slash(ans[0],'-')
    return ans
def type2(val):
    type2 = re.compile(r'\d{1,2}[ ](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ ]\d{2,4}')
    ans = re.findall(type2,val)
    return ans[0]
def type3(val):
    type3 = re.compile(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:\.| )+\d{1,2}(?:,| )+\d{,4}')
    ans = re.findall(type3,val)
    return ans[0]
def type4(val):
    type4 = re.compile(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:,| )+\d{4}')
    ans = re.findall(type4,val)
    return ans[0]
def type5(val):
    type5 = re.compile(r'\d{1,2}[/]\d{2,4}')
    ans = re.findall(type5,val)
    return ans[0]
def type6(val):
    type6 = re.compile(r'\d{4}')
    ans = re.findall(type6,val)
    return ans[0]

In [6]:
def date_sorter():
    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)
    df = pd.Series(doc)
    # 03/25/93
    df1 = pd.Series(df[0:125].apply(lambda x: type1(x))) # index 0 to 124
    df1.iloc[72] = '7/11/77'    #72 issue '7/11/77'
    df1 = df1.apply(lambda x: add_nineteen(x))
    # 24 Jan 2001
    df2 = pd.Series(df[125:194].apply(lambda x: type2(x))) # index 125 to 193
    df2 = df2.apply(lambda x: add_slash(x,' '))
    df2 = df2.apply(lambda x: t2_months(x))
    df2 = df2.apply(lambda x: add_nineteen(x))
    # April 11, 1990
    df3 = pd.Series(df[194:228].apply(lambda x: type3(x))) # index 194 to 227
    df3 = df3.apply(lambda x: clean_punct(x))
    df3 = df3.apply(lambda x: add_slash(x,' '))
    df3 = df3.apply(lambda x: t3_months(x))
    df3 = df3.apply(lambda x: add_nineteen(x))
    # September 1985
    df4 = pd.Series(df[228:343].apply(lambda x: type4(x))) # index 228 to 342
    df4 = df4.apply(lambda x: clean_punct(x))
    df4 = df4.apply(lambda x: add_slash(x,' '))
    df4 = df4.apply(lambda x: t4_months(x))
    df4 = df4.apply(lambda x: add_nineteen(x))
    # 6/1998
    df5 = pd.Series(df[343:455].apply(lambda x: type5(x))) # index 343 to 454
    df5 = df5.apply(lambda x: t5_months(x))
    df5 = df5.apply(lambda x: add_nineteen(x))
    # 1984
    df6 = pd.Series(df[455:].apply(lambda x: type6(x))) # index 455 to 500
    df6 = df6.apply(lambda x: t6_months(x))
    df_tot = pd.concat([df1,df2,df3,df4,df5,df6])
    datetimes = pd.to_datetime(df_tot, format="%m/%d/%Y")
    sorted_dates = datetimes.sort_values()
    answer = sorted_dates.reset_index()['index']
    return answer    